<a href="https://colab.research.google.com/github/MarriRohan/HULK-FOOD-DELIVERY-APP/blob/main/HFDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
"""
hulk_food.py
Hulk Food Delivery - simple Python backend prototype for gym users.
Features:
- Compute protein and calorie targets for "cutting" and "bulking"
- Suggest a daily meal plan (breakfast, lunch, dinner, snacks) from a menu
- Greedy selection to reach protein targets within calorie window
- Easy to extend (add meals, change targets, add ML later)

Run: python hulk_food.py
"""

from typing import List, Dict, Tuple
import random

# ---------- Example meal database ----------
# Each item: name, calories, protein (g), carbs (g), fats (g), tags
# NOTE: Price is added in a later cell. This definition is the original for function compatibility.
MEALS = [
    ("Egg white omelette w/ spinach & oats", 350, 32, 30, 10, ["breakfast", "high-protein"]),
    ("Greek yogurt + whey + berries", 300, 35, 25, 6, ["breakfast", "snack"]),
    ("Chicken breast (200g) + brown rice (150g) + broccoli", 620, 60, 60, 8, ["lunch", "high-protein"]),
    ("Tuna salad (canned tuna 150g) + greens + olive oil", 420, 45, 10, 18, ["lunch", "low-carb"]),
    ("Grilled salmon 180g + sweet potato 200g + asparagus", 650, 50, 55, 22, ["dinner", "omega-3"]),
    ("Lean beef (200g) + quinoa 120g + mixed veg", 700, 62, 65, 18, ["dinner", "high-protein"]),
    ("Cottage cheese 200g + walnuts", 360, 28, 12, 22, ["snack", "slow-protein"]),
    ("Protein shake (whey) + banana", 250, 30, 30, 3, ["snack"]),
    ("Turkey wrap (turkey 120g) + whole wheat wrap + lettuce", 420, 35, 45, 8, ["lunch", "portable"]),
    ("Tofu stir-fry with brown rice", 540, 28, 70, 12, ["vegan", "lunch"]),
    ("Peanut butter sandwich (2 tbsp + whole grain)", 410, 18, 40, 20, ["snack", "bulking-friendly"]),
    ("Protein pancakes (oats + whey + egg)", 480, 40, 55, 10, ["breakfast", "bulking-friendly"]),
]

# ---------- Helper functions ----------
def round2(x):
    return round(x, 2)

def daily_targets(weight_kg: float, goal: str, activity_multiplier: float = 1.2) -> Dict[str, float]:
    """
    Compute daily calorie and protein targets.
    - weight_kg: user's body weight in kilograms
    - goal: 'cutting' or 'bulking'
    - activity_multiplier: baseline multiplier for daily calorie need (user activity)
    Returns dict: {calories_target, protein_target_g, calorie_min, calorie_max}
    """
    # Basal estimate using Mifflin-St Jeor is complex; keep simple: maintenance cal = 24 kcal/kg * activity_mult
    maintenance = 24 * weight_kg * activity_multiplier
    # Protein:
    # - Cutting: 2.0 g/kg (preserve muscle)
    # - Bulking: 1.8 g/kg (sufficient for growth)
    if goal == "cutting":
        protein_g = 2.0 * weight_kg
        calories_target = maintenance * 0.8  # 20% deficit
    elif goal == "bulking":
        protein_g = 1.8 * weight_kg
        calories_target = maintenance * 1.15  # 15% surplus
    else:
        raise ValueError("goal must be 'cutting' or 'bulking'")
    # Allow a small calorie window (±7%)
    cal_min = calories_target * 0.93
    cal_max = calories_target * 1.07
    return {
        "calories_target": round2(calories_target),
        "protein_target_g": round2(protein_g),
        "cal_min": round2(cal_min),
        "cal_max": round2(cal_max),
    }

def filter_meals_by_tag(meals, tag):
    return [m for m in meals if tag in m[5]]

# Simple greedy selector: maximize protein per calorie, choose meals until hitting protein target without exceeding calorie_max too much
def build_daily_plan(meals: List[Tuple], protein_target: float, cal_min: float, cal_max: float) -> Tuple[List[Tuple], Dict]:
    """
    Returns (selected meals list, stats)
    Greedy approach:
    - Score by protein_per_cal = protein / calories
    - Pick top scoring meals while total_cal <= cal_max and protein < protein_target
    - If after selection protein < target, try to add highest-protein items even if calories go slightly over max
    """
    meals_copy = meals[:]
    # shuffle to add variety if equal scores
    random.shuffle(meals_copy)
    # Modify to unpack only the first 6 elements (excluding price if present)
    scored = sorted(meals_copy, key=lambda x: (x[2] / x[1]) if x[1] > 0 else 0, reverse=True)  # protein/calorie
    selected = []
    total_cal = 0
    total_protein = 0
    # First pass: pick items until protein target reached or calories close to max
    for item in scored:
        # Unpack only the first 6 elements
        name, cal, protein, carbs, fats, tags = item[:6]
        if total_cal + cal <= cal_max and total_protein < protein_target:
            selected.append(item) # Append the original tuple including price
            total_cal += cal
            total_protein += protein
    # If still under protein target, try to add top-protein items (may go slightly over cal_max)
    if total_protein < protein_target:
        # Sort by protein, considering only the protein value at index 2
        remaining_sorted = sorted(meals_copy, key=lambda x: x[2], reverse=True)
        for item in remaining_sorted:
            if item in selected:
                continue
            # Unpack only the first 6 elements for calculation
            name, cal, protein, carbs, fats, tags = item[:6]
            selected.append(item) # Append the original tuple including price
            total_cal += cal
            total_protein += protein
            if total_protein >= protein_target:
                break
    # Post-process: ensure not too many meals (cap to 6 items)
    if len(selected) > 6:
        selected = selected[:6]
        # Recalculate totals based on the clipped list
        total_cal = sum(m[1] for m in selected)
        total_protein = sum(m[2] for m in selected)

    stats = {
        "total_calories": round2(total_cal),
        "total_protein_g": round2(total_protein),
        "protein_target_g": round2(protein_target),
        "cal_min": round2(cal_min),
        "cal_max": round2(cal_max),
    }
    return selected, stats

def split_into_meals(selected: List[Tuple]) -> Dict[str, List[Tuple]]:
    """
    Map selected items roughly into breakfast/lunch/dinner/snacks based on tags and protein distribution.
    This is heuristic-based for demo.
    """
    plan = {"breakfast": [], "lunch": [], "dinner": [], "snacks": []}
    for item in selected:
        # Unpack only the first 6 elements for tag and name checks
        name, cal, protein, carbs, fats, tags = item[:6]
        if "breakfast" in tags or "pancake" in name.lower() or len(plan["breakfast"]) < 1 and random.random() < 0.2:
            plan["breakfast"].append(item)
        elif "dinner" in tags or "omega-3" in tags or "salmon" in name.lower():
            plan["dinner"].append(item)
        elif "lunch" in tags or "wrap" in name.lower() or "chicken" in name.lower():
            plan["lunch"].append(item)
        else:
            # fallback to snacks or wherever short
            if len(plan["snacks"]) < 2:
                plan["snacks"].append(item)
            else:
                plan["lunch"].append(item)
    return plan

# ---------- CLI / Example ----------
def pretty_print_plan(plan_map: Dict[str, List[Tuple]], stats: Dict):
    print("\n--- Hulk Food Daily Plan ---")
    print(f"Calories target window: {stats['cal_min']} - {stats['cal_max']} kcal")
    print(f"Daily protein target: {stats['protein_target_g']} g")
    print(f"Planned calories: {stats['total_calories']} kcal | planned protein: {stats['total_protein_g']} g\n")
    for meal_time in ["breakfast", "lunch", "dinner", "snacks"]:
        items = plan_map.get(meal_time, [])
        if not items:
            continue
        print(f"{meal_time.upper()}:")
        for name, cal, protein, carbs, fats, tags in items:
            print(f"  - {name}  ({cal} kcal, {protein}g protein, {carbs}g carbs, {fats}g fats)")
        print()

def run_demo():
    print("Welcome to Hulk Food — Meal suggester prototype for gym bros.")
    # Example: ask user or use demo values
    try:
        weight = float(input("Enter body weight in kg (e.g., 75): ").strip())
    except Exception:
        weight = 75.0
        print("Invalid input — using default 75 kg.")
    goal = input("Goal? Type 'cutting' or 'bulking' (default: cutting): ").strip().lower()
    if goal not in ("cutting", "bulking"):
        goal = "cutting"
        print("Using default goal: cutting")
    activity = input("Activity level: low/moderate/high (default moderate): ").strip().lower()
    if activity == "low":
        act_mult = 1.1
    elif activity == "high":
        act_mult = 1.35
    else:
        act_mult = 1.2
    targets = daily_targets(weight, goal, activity_multiplier=act_mult)
    selected, stats = build_daily_plan(MEALS, targets["protein_target_g"], targets["cal_min"], targets["cal_max"])
    plan_map = split_into_meals(selected)
    # Merge stats with targets for printing
    stats.update({
        "cal_min": targets["cal_min"],
        "cal_max": targets["cal_max"],
        "protein_target_g": targets["protein_target_g"]
    })
    pretty_print_plan(plan_map, stats)

if __name__ == "__main__":
    run_demo()

Welcome to Hulk Food — Meal suggester prototype for gym bros.
Enter body weight in kg (e.g., 75): 72
Goal? Type 'cutting' or 'bulking' (default: cutting): cutting
Activity level: low/moderate/high (default moderate): moderate

--- Hulk Food Daily Plan ---
Calories target window: 1542.76 - 1775.0 kcal
Daily protein target: 144.0 g
Planned calories: 1590 kcal | planned protein: 170 g

BREAKFAST:
  - Protein shake (whey) + banana  (250 kcal, 30g protein, 30g carbs, 3g fats)
  - Greek yogurt + whey + berries  (300 kcal, 35g protein, 25g carbs, 6g fats)

LUNCH:
  - Tuna salad (canned tuna 150g) + greens + olive oil  (420 kcal, 45g protein, 10g carbs, 18g fats)
  - Chicken breast (200g) + brown rice (150g) + broccoli  (620 kcal, 60g protein, 60g carbs, 8g fats)



In [16]:
# Add this price field to MEALS (last number is price in USD for demo)
MEALS = [
    ("Egg white omelette w/ spinach & oats", 350, 32, 30, 10, ["breakfast", "high-protein"], 5.99),
    ("Greek yogurt + whey + berries", 300, 35, 25, 6, ["breakfast", "snack"], 4.50),
    ("Chicken breast (200g) + brown rice (150g) + broccoli", 620, 60, 60, 8, ["lunch", "high-protein"], 8.99),
    ("Tuna salad (canned tuna 150g) + greens + olive oil", 420, 45, 10, 18, ["lunch", "low-carb"], 7.50),
    ("Grilled salmon 180g + sweet potato 200g + asparagus", 650, 50, 55, 22, ["dinner", "omega-3"], 10.50),
    ("Lean beef (200g) + quinoa 120g + mixed veg", 700, 62, 65, 18, ["dinner", "high-protein"], 9.99),
    ("Cottage cheese 200g + walnuts", 360, 28, 12, 22, ["snack", "slow-protein"], 4.20),
    ("Protein shake (whey) + banana", 250, 30, 30, 3, ["snack"], 3.50),
    ("Turkey wrap (turkey 120g) + whole wheat wrap + lettuce", 420, 35, 45, 8, ["lunch", "portable"], 6.99),
    ("Tofu stir-fry with brown rice", 540, 28, 70, 12, ["vegan", "lunch"], 6.50),
    ("Peanut butter sandwich (2 tbsp + whole grain)", 410, 18, 40, 20, ["snack", "bulking-friendly"], 3.80),
    ("Protein pancakes (oats + whey + egg)", 480, 40, 55, 10, ["breakfast", "bulking-friendly"], 5.50),
]

# Modify pretty_print_plan to also number meals
def pretty_print_plan(plan_map: Dict[str, List[Tuple]], stats: Dict):
    print("\n--- Hulk Food Daily Plan ---")
    print(f"Calories target window: {stats['cal_min']} - {stats['cal_max']} kcal")
    print(f"Daily protein target: {stats['protein_target_g']} g")
    print(f"Planned calories: {stats['total_calories']} kcal | planned protein: {stats['total_protein_g']} g\n")

    meal_counter = 1
    meal_index_map = {}  # number -> meal tuple

    for meal_time in ["breakfast", "lunch", "dinner", "snacks"]:
        items = plan_map.get(meal_time, [])
        if not items:
            continue
        print(f"{meal_time.upper()}:")
        for item in items:
            name, cal, protein, carbs, fats, tags, price = item
            print(f"  [{meal_counter}] {name}  (${price:.2f}) "
                  f"({cal} kcal, {protein}g protein, {carbs}g carbs, {fats}g fats)")
            meal_index_map[meal_counter] = item
            meal_counter += 1
        print()
    return meal_index_map

def add_to_cart(meal_index_map):
    cart = []
    while True:
        choice = input("Enter meal number to add to cart (or 'done' to finish): ").strip().lower()
        if choice == "done":
            break
        try:
            num = int(choice)
            if num in meal_index_map:
                cart.append(meal_index_map[num])
                print(f"✅ Added '{meal_index_map[num][0]}' to cart.")
            else:
                print("❌ Invalid meal number.")
        except ValueError:
            print("❌ Please enter a valid number or 'done'.")
    return cart

def checkout(cart):
    if not cart:
        print("🛒 Cart is empty. Nothing to checkout.")
        return
    total_price = sum(item[6] for item in cart)
    total_cal = sum(item[1] for item in cart)
    total_protein = sum(item[2] for item in cart)
    print("\n--- Checkout ---")
    for item in cart:
        print(f"- {item[0]} (${item[6]:.2f})")
    print(f"Total: ${total_price:.2f}")
    print(f"Total Calories: {total_cal} kcal")
    print(f"Total Protein: {total_protein} g")
    print("💪 Thank you for ordering from Hulk Food!")

def run_demo():
    print("Welcome to Hulk Food — Meal suggester + cart system.")
    try:
        weight = float(input("Enter body weight in kg (e.g., 75): ").strip())
    except Exception:
        weight = 75.0
        print("Invalid input — using default 75 kg.")
    goal = input("Goal? Type 'cutting' or 'bulking' (default: cutting): ").strip().lower()
    if goal not in ("cutting", "bulking"):
        goal = "cutting"
        print("Using default goal: cutting")
    activity = input("Activity level: low/moderate/high (default moderate): ").strip().lower()
    if activity == "low":
        act_mult = 1.1
    elif activity == "high":
        act_mult = 1.35
    else:
        act_mult = 1.2
    targets = daily_targets(weight, goal, activity_multiplier=act_mult)
    selected, stats = build_daily_plan(MEALS, targets["protein_target_g"], targets["cal_min"], targets["cal_max"])
    plan_map = split_into_meals(selected)
    stats.update({
        "cal_min": targets["cal_min"],
        "cal_max": targets["cal_max"],
        "protein_target_g": targets["protein_target_g"]
    })

    meal_index_map = pretty_print_plan(plan_map, stats)
    cart = add_to_cart(meal_index_map)
    checkout(cart)

if __name__ == "__main__":
    run_demo()



Welcome to Hulk Food — Meal suggester + cart system.
Enter body weight in kg (e.g., 75): 72
Goal? Type 'cutting' or 'bulking' (default: cutting): cutting
Activity level: low/moderate/high (default moderate): moderate

--- Hulk Food Daily Plan ---
Calories target window: 1542.76 - 1775.0 kcal
Daily protein target: 144.0 g
Planned calories: 1590 kcal | planned protein: 170 g

BREAKFAST:
  [1] Greek yogurt + whey + berries  ($4.50) (300 kcal, 35g protein, 25g carbs, 6g fats)

LUNCH:
  [2] Tuna salad (canned tuna 150g) + greens + olive oil  ($7.50) (420 kcal, 45g protein, 10g carbs, 18g fats)
  [3] Chicken breast (200g) + brown rice (150g) + broccoli  ($8.99) (620 kcal, 60g protein, 60g carbs, 8g fats)

SNACKS:
  [4] Protein shake (whey) + banana  ($3.50) (250 kcal, 30g protein, 30g carbs, 3g fats)

Enter meal number to add to cart (or 'done' to finish): 2
✅ Added 'Tuna salad (canned tuna 150g) + greens + olive oil' to cart.
Enter meal number to add to cart (or 'done' to finish): done

---